**Disclaimer**: This agent is not intended as financial advice.  It is for informational and entertainment purposes only.  Do your own due diligence.

Hi there 👋 - the goal of this code is to create a financial agent that can analyze stocks like **Warren Buffett**.  

The underlying financial data comes from [financialdatasets.ai](https://www.financialdatasets.ai), which is a stock market API that provides:


*   income statements
*   balance sheets
*   cash flow statements
*   sec filings
*   stock prices

Credit: Build Fast With AI Session.

# 0. Setup and installation

In [ ]:
!pip install -U --quiet langgraph langchain_community langchain_anthropic langsmith langchain_openai

In [ ]:
import getpass
import os

In [ ]:

# Set your Anthropic API key
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

In [ ]:
# You can get an API key here https://financialdatasets.ai/
os.environ["FINANCIAL_DATASETS_API_KEY"] = getpass.getpass()

In [ ]:
# Set your Tavily API key from https://tavily.com/
os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [ ]:
# You can create an API key here https://smith.langchain.com/settings
os.environ["LANGCHAIN_TRACING_V2"] = "True"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

# 1. Define tools for financial metric computations

In [ ]:
from langchain_core.tools import tool


@tool
def roe(
    net_income: float,
    equity: float,
) -> float:
    """
    Computes the return on equity (ROE) for a given company.
    Use this function to evaluate the profitability of a company.
    """
    return net_income / equity


@tool
def roic(
    operating_income: float,
    total_debt: float,
    equity: float,
    cash_and_equivalents: float,
    tax_rate: float = 0.35,
) -> float:
    """
    Computes the return on invested capital (ROIC) for a given company.
    Use this function to evaluate the efficiency of a company in generating returns from its capital.
    """
    net_operating_profit_after_tax = operating_income * (1 - tax_rate)
    invested_capital = total_debt + equity - cash_and_equivalents
    return net_operating_profit_after_tax / invested_capital


@tool
def owner_earnings(
    net_income: float,
    depreciation_amortization: float = 0.0,
    capital_expenditures: float = 0.0
):
    """
    Calculates the owner earnings for a company based on the net income, depreciation/amortization, and capital expenditures.
    """
    return net_income + depreciation_amortization - capital_expenditures


@tool
def intrinsic_value(
    free_cash_flow: float,
    growth_rate: float = 0.05,
    discount_rate: float = 0.10,
    terminal_growth_rate: float = 0.02,
    num_years: int = 5,
) -> float:
    """
    Computes the discounted cash flow (DCF) for a given company based on the current free cash flow.
    Use this function to calculate the intrinsic value of a stock.
    """
    # Estimate the future cash flows based on the growth rate
    cash_flows = [free_cash_flow * (1 + growth_rate) ** i for i in range(num_years)]

    # Calculate the present value of projected cash flows
    present_values = []
    for i in range(num_years):
        present_value = cash_flows[i] / (1 + discount_rate) ** (i + 1)
        present_values.append(present_value)

    # Calculate the terminal value
    terminal_value = cash_flows[-1] * (1 + terminal_growth_rate) / (discount_rate - terminal_growth_rate)
    terminal_present_value = terminal_value / (1 + discount_rate) ** num_years

    # Sum up the present values and terminal value
    dcf_value = sum(present_values) + terminal_present_value

    return dcf_value

@tool
def percentage_change(start: float, end: float):
    """
    Calculate the percentage change between two floats, start and end.

    :param start: The starting value
    :param end: The end value
    :return: The percentage change as a float
    """
    if start == 0:
        raise ValueError("Start cannot be zero")

    price_change = end - start
    percentage_change = (price_change / start) * 100

    return round(percentage_change, 2)

# 2. Define tool for getting stock prices

In [ ]:
import os
from typing import Dict, Union
from langchain.pydantic_v1 import BaseModel, Field
import requests

class GetPricesInput(BaseModel):
    ticker: str = Field(..., description="The ticker of the stock.")
    start_date: str = Field(..., description="The start of the price time window. Either a date with the format YYYY-MM-DD or a millisecond timestamp.")
    end_date: str = Field(..., description="The end of the aggregate time window. Either a date with the format YYYY-MM-DD or a millisecond timestamp.")
    interval: str = Field(
        default="day",
        description="The time interval of the prices. Valid values are second', 'minute', 'day', 'week', 'month', 'quarter', 'year'.",
    )
    interval_multiplier: int = Field(
        default=1,
        description="The multiplier for the interval. For example, if interval is 'day' and interval_multiplier is 1, the prices will be daily. "
                    "If interval is 'minute' and interval_multiplier is 5, the prices will be every 5 minutes.",
    )
    limit: int = Field(
        default=5000,
        description="The maximum number of prices to return. The default is 5000 and the maximum is 50000.",
    )

@tool("get_prices", args_schema=GetPricesInput, return_direct=True)
def get_prices(ticker: str, start_date: str, end_date: str, interval: str, interval_multiplier: int = 1, limit: int = 5000) -> Union[Dict, str]:
    """
    Get prices for a ticker over a given date range and interval.
    """

    api_key = os.environ.get("FINANCIAL_DATASETS_API_KEY")
    if not api_key:
        raise ValueError("Missing FINANCIAL_DATASETS_API_KEY.")
    url = (
        f"https://api.financialdatasets.ai/prices"
        f"?ticker={ticker}"
        f"&start_date={start_date}"
        f"&end_date={end_date}"
        f"&interval={interval}"
        f"&interval_multiplier={interval_multiplier}"
        f"&limit={limit}"
    )

    try:
        response = requests.get(url, headers={'X-API-Key': api_key})
        data = response.json()
        return data
    except Exception as e:
        return {"ticker": ticker, "prices": [], "error": str(e)}


# 3. Define Financial Search tool

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List, Optional
from datetime import date
from enum import Enum


class LineItem(str, Enum):
    # Income Statement fields
    consolidated_income = "consolidated_income"
    cost_of_revenue = "cost_of_revenue"
    dividends_per_common_share = "dividends_per_common_share"
    earnings_per_share = "earnings_per_share"
    earnings_per_share_diluted = "earnings_per_share_diluted"
    ebit = "ebit"
    ebit_usd = "ebit_usd"
    earnings_per_share_usd = "earnings_per_share_usd"
    gross_profit = "gross_profit"
    income_tax_expense = "income_tax_expense"
    interest_expense = "interest_expense"
    net_income = "net_income"
    net_income_common_stock = "net_income_common_stock"
    net_income_common_stock_usd = "net_income_common_stock_usd"
    net_income_discontinued_operations = "net_income_discontinued_operations"
    net_income_non_controlling_interests = "net_income_non_controlling_interests"
    operating_expense = "operating_expense"
    operating_income = "operating_income"
    preferred_dividends_impact = "preferred_dividends_impact"
    research_and_development = "research_and_development"
    revenue = "revenue"
    revenue_usd = "revenue_usd"
    selling_general_and_administrative_expenses = "selling_general_and_administrative_expenses"
    weighted_average_shares = "weighted_average_shares"
    weighted_average_shares_diluted = "weighted_average_shares_diluted"

    # Balance Sheet fields
    accumulated_other_comprehensive_income = "accumulated_other_comprehensive_income"
    cash_and_equivalents = "cash_and_equivalents"
    cash_and_equivalents_usd = "cash_and_equivalents_usd"
    current_assets = "current_assets"
    current_debt = "current_debt"
    current_investments = "current_investments"
    current_liabilities = "current_liabilities"
    deferred_revenue = "deferred_revenue"
    deposit_liabilities = "deposit_liabilities"
    goodwill_and_intangible_assets = "goodwill_and_intangible_assets"
    inventory = "inventory"
    investments = "investments"
    non_current_assets = "non_current_assets"
    non_current_debt = "non_current_debt"
    non_current_investments = "non_current_investments"
    non_current_liabilities = "non_current_liabilities"
    outstanding_shares = "outstanding_shares"
    property_plant_and_equipment = "property_plant_and_equipment"
    retained_earnings = "retained_earnings"
    shareholders_equity = "shareholders_equity"
    shareholders_equity_usd = "shareholders_equity_usd"
    tax_assets = "tax_assets"
    tax_liabilities = "tax_liabilities"
    total_assets = "total_assets"
    total_debt = "total_debt"
    total_debt_usd = "total_debt_usd"
    total_liabilities = "total_liabilities"
    trade_and_non_trade_payables = "trade_and_non_trade_payables"
    trade_and_non_trade_receivables = "trade_and_non_trade_receivables"

    # Cash Flow Statement fields
    business_acquisitions_and_disposals = "business_acquisitions_and_disposals"
    capital_expenditure = "capital_expenditure"
    change_in_cash_and_equivalents = "change_in_cash_and_equivalents"
    depreciation_and_amortization = "depreciation_and_amortization"
    dividends_and_other_cash_distributions = "dividends_and_other_cash_distributions"
    effect_of_exchange_rate_changes = "effect_of_exchange_rate_changes"
    investment_acquisitions_and_disposals = "investment_acquisitions_and_disposals"
    issuance_or_purchase_of_equity_shares = "issuance_or_purchase_of_equity_shares"
    issuance_or_repayment_of_debt_securities = "issuance_or_repayment_of_debt_securities"
    net_cash_flow_from_financing = "net_cash_flow_from_financing"
    net_cash_flow_from_investing = "net_cash_flow_from_investing"
    net_cash_flow_from_operations = "net_cash_flow_from_operations"
    share_based_compensation = "share_based_compensation"


class SearchLineItemsInput(BaseModel):
    tickers: List[str] = Field(..., description="List of stock tickers to search for.")
    line_items: List[LineItem] = Field(..., description="List of financial line items to retrieve.")
    period: str = Field(
        default="ttm",
        description="The time period for the financial data. Valid values are 'annual', 'quarterly', or 'ttm' (trailing twelve months)."
    )
    limit: int = Field(
        default=1,
        description="The maximum number of results to return per ticker. Must be a positive integer."
    )
    start_date: Optional[date] = Field(
        None,
        description="The start date for the financial data in YYYY-MM-DD format."
    )
    end_date: Optional[date] = Field(
        None,
        description="The end date for the financial data in YYYY-MM-DD format."
    )

    class Config:
        schema_extra = {
            "example": {
                "tickers": ["AAPL", "GOOGL"],
                "line_items": ["revenue", "net_income", "total_assets"],
                "period": "annual",
                "limit": 5,
                "start_date": "2020-01-01",
                "end_date": "2024-09-01"
            }
        }


In [ ]:
import os
from typing import Dict, Union, List

import requests
from langchain_core.tools import tool

@tool("search-line-items", args_schema=SearchLineItemsInput, return_direct=True)
def search_line_items(
    tickers: List[str],
    line_items: List[str],
    period: str = "ttm",
    limit: int = 1,
    start_date: str = None,
    end_date: str = None
) -> Union[Dict, str]:
    """
    Search for specific financial line items across multiple company tickers over a specified time period.

    Note: This tool accesses real financial data and should be used when specific, factual financial line items are required.
    """
    BASE_URL = "https://api.financialdatasets.ai/"

    api_key = os.environ.get("FINANCIAL_DATASETS_API_KEY")
    if not api_key:
        raise ValueError("Missing FINANCIAL_DATASETS_API_KEY.")

    url = f"{BASE_URL}financials/search/line-items"

    payload = {
        "tickers": tickers,
        "line_items": line_items,
        "period": period,
        "limit": limit
    }

    if start_date:
        payload["start_date"] = start_date
    if end_date:
        payload["end_date"] = end_date

    try:
        response = requests.post(
            url,
            json=payload,
            headers={'X-API-Key': api_key, 'Content-Type': 'application/json'}
        )
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        return {"search_results": [], "error": str(e)}


# 4. Define Web Search tool

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import List, Optional


class TavilySearchInput(BaseModel):
    query: str = Field(..., description="The search query you want to execute with Tavily.")
    search_depth: Optional[str] = Field(
        default="basic",
        description="The depth of the search. It can be 'basic' or 'advanced'."
    )
    topic: Optional[str] = Field(
        default="general",
        description="The category of the search. Currently supports 'general' and 'news'."
    )
    days: Optional[int] = Field(
        default=3,
        description="The number of days back from the current date to include in the search results. Only available for 'news' topic."
    )
    max_results: Optional[int] = Field(
        default=5,
        description="The maximum number of search results to return."
    )
    include_images: Optional[bool] = Field(
        default=False,
        description="Include a list of query-related images in the response."
    )
    include_image_descriptions: Optional[bool] = Field(
        default=False,
        description="When include_images is True, adds descriptive text for each image."
    )
    include_answer: Optional[bool] = Field(
        default=False,
        description="Include a short answer to original query."
    )
    include_raw_content: Optional[bool] = Field(
        default=False,
        description="Include the cleaned and parsed HTML content of each search result."
    )
    include_domains: Optional[List[str]] = Field(
        default=[],
        description="A list of domains to specifically include in the search results."
    )
    exclude_domains: Optional[List[str]] = Field(
        default=[],
        description="A list of domains to specifically exclude from the search results."
    )

    class Config:
        schema_extra = {
            "example": {
                "query": "Latest advancements in AI",
                "api_key": "your-api-key-here",
                "search_depth": "advanced",
                "topic": "news",
                "days": 7,
                "max_results": 10,
                "include_images": True,
                "include_image_descriptions": True,
                "include_answer": True,
                "include_raw_content": False,
                "include_domains": ["techcrunch.com", "wired.com"],
                "exclude_domains": ["example.com"]
            }
        }


In [ ]:
import os
from typing import Dict, Union

import requests
from langchain_core.tools import tool

@tool("search-web", args_schema=TavilySearchInput, return_direct=True)
def search_web(
    query: str,
    search_depth: str = "basic",
    topic: str = "general",
    days: int = 3,
    max_results: int = 3,
    include_images: bool = False,
    include_answer: bool = False,
    include_raw_content: bool = False,
    include_domains: list = None,
    exclude_domains: list = None
) -> Union[Dict, str]:
    """
    Perform a web search using the Tavily API.

    This tool accesses real-time web data, news, articles and should be used when up-to-date information from the internet is required.
    """
    TAVILY_BASE_URL = "https://api.tavily.com"

    api_key = os.environ.get("TAVILY_API_KEY")
    if not api_key:
        raise ValueError("Missing TAVILY_API_KEY in environment variables.")

    payload = {
        "api_key": api_key,
        "query": query,
        "search_depth": search_depth,
        "topic": topic,
        "days": days if topic == "news" else None,
        "max_results": max_results,
        "include_images": include_images,
        "include_answer": include_answer,
        "include_raw_content": include_raw_content
    }

    if include_domains:
        payload["include_domains"] = include_domains
    if exclude_domains:
        payload["exclude_domains"] = exclude_domains

    try:
        response = requests.post(
            f"{TAVILY_BASE_URL}/search",
            json=payload,
            headers={'Content-Type': 'application/json'}
        )
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}


if __name__ == "__main__":
    # Example usage of the tool
    search_web("Latest advancements in AI")

# 5. Set up the LLM

In [ ]:
from langgraph.prebuilt import ToolNode

from langchain_community.tools import IncomeStatements, BalanceSheets, CashFlowStatements
from langchain_community.utilities.financial_datasets import FinancialDatasetsAPIWrapper

# Create the tools
api_wrapper = FinancialDatasetsAPIWrapper()
integration_tools = [
    IncomeStatements(api_wrapper=api_wrapper),
    BalanceSheets(api_wrapper=api_wrapper),
    CashFlowStatements(api_wrapper=api_wrapper),
]

local_tools = [intrinsic_value, roe, roic, owner_earnings, get_prices, percentage_change, search_line_items, search_web]
tools = integration_tools + local_tools

tool_node = ToolNode(tools)

In [ ]:
from langchain.tools.render import format_tool_to_openai_function
from langchain_anthropic.chat_models import ChatAnthropic
from langchain_openai.chat_models import ChatOpenAI

sonnet_model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0).bind_tools(tools)

In [ ]:
import datetime

system_prompt = f"""
You are an AI financial agent with expertise in analyzing businesses using methods similar to those of Warren Buffett. Your task is to provide short, accurate, and concise answers to questions about company financials and performance.

You use financial tools to answer the questions.  The tools give you access to data sources like income statements, stock prices, etc.

Here are a few example questions and answers:

# Example 1:
question: What was NVDA's net income for the fiscal year 2023?
answer: The net income for NVDA in 2023 was $2.8 billion.

# Example 2:
question: How did NVDA's gross profit in 2023 compare to its gross profit in 2022?
answer: In 2023, NVDA's gross profit increased by 12% compared to 2022.

# Example 3:
question: What was NVDA's revenue for the first quarter of 2024?,
answer: NVDA's revenue for the first quarter of 2024 was $5.6 billion.

Analyze these examples carefully. Notice how the answers are concise, specific, and directly address the questions asked. They provide precise financial figures and, when applicable, comparative analysis.

When answering questions:
1. Focus on providing accurate financial data and insights.
2. Use specific numbers and percentages when available.
3. Make comparisons between different time periods if relevant.
4. Keep your answers short, concise, and to the point.

Important: You must be short and concise with your answers.

The current date is {datetime.date.today().strftime("%Y-%m-%d")}
"""

# 6. Define the agent state

In [ ]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

# 7. Define the nodes

In [ ]:
from typing import Literal
from langgraph.graph import END, StateGraph, MessagesState
from langchain_core.messages import SystemMessage

# Define the function that determines whether to continue or not
def should_continue(state: MessagesState) -> Literal["tools", END]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "tools" node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, we stop (reply to the user)
    return END

# Define the function that calls the model
def call_agent(state: MessagesState):
    prompt = SystemMessage(
        content=system_prompt
    )
    # Get the messages
    messages = state['messages']

    # Check if first message in messages is the prompt
    if messages and messages[0].content != system_prompt:
        # Add the prompt to the start of the message
        messages.insert(0, prompt)

    # We return a list, because this will get added to the existing list
    return {"messages": [sonnet_model.invoke(messages)]}

def call_output(state: MessagesState):
    prompt = SystemMessage(
        content=system_prompt
    )
    # Get the messages
    messages = state['messages']

    # Check if first message in messages is the prompt
    if messages and messages[0].content != system_prompt:
        # Add the prompt to the start of the message
        messages.insert(0, prompt)
    return {"messages": [sonnet_model.invoke(messages)]}


# 8. Define the graph

In [ ]:
# Define a new graph
workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_agent)
workflow.add_node("tools", tool_node)
workflow.add_node("output", call_output)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    {"tools": "tools", END: END}
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge("tools", "output")

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable.
# Note that we're (optionally) passing the memory when compiling the graph
app = workflow.compile()

# 9. Run the financial agent

In [ ]:
from langchain_core.messages import HumanMessage

# Use the Runnable
final_state = app.invoke(
    {"messages": [HumanMessage(content="What is the latest news for AAPL?")]},
    config={"configurable": {"thread_id": 42}}
)
output = final_state["messages"][-1].content
print(' '.join(output.split()))

Based on the latest news for Apple (AAPL), here are the key points: 1. Record high stock price: Apple's stock reached an all-time high of $237.49 on October 15, 2024. 2. Market capitalization: The company's market cap hit a record $3.6 trillion. 3. AI-driven growth: The stock surge is attributed to optimism surrounding the release of AI-focused products, including: - The iPhone 16 with AI features - Apple Intelligence, an AI-focused iPhone operating software, launching on October 28, 2024 4. Upcoming earnings: Apple is set to report its quarterly earnings on October 31, 2024. 5. Strong performance: Apple's stock has risen 37% over the past six months, outperforming the Nasdaq 100 Index. These developments indicate positive market sentiment towards Apple's AI initiatives and continued growth prospects.


In [ ]:
final_state = app.invoke(
    {"messages": [HumanMessage(content="What was AAPL's revenue and net income in FY 2023?")]},
    config={"configurable": {"thread_id": 42}}
)
output = final_state["messages"][-1].content
print(' '.join(output.split()))

Based on the income statement data for Apple (AAPL) in fiscal year 2023: AAPL's revenue was $383.29 billion and net income was $97.00 billion.
